In [1]:
# リスト 3.1.1
# PythonからCaboChaの呼び出し（トークン列の取得）
# -----------------------------------------------------------------------------------
# 目的:
#   ・日本語係り受け解析器 CaboCha を Python から呼び出し、文を
#     - 文節（chunk, ぶんせつ）単位に分割し
#     - 各文節に含まれる形態素トークン（token）を列挙する
#     という最小例を示す。
#
# 理論メモ（挙動は変えず背景のみ補足）:
#   ・CaboCha は「形態素解析（通常は MeCab）」→「文節分割」→「係り受け推定」のパイプラインを持つ。
#   ・本コードは “係り受け” 自体は出さず、文節ごとの“形態素トークン列”だけを出力している。
#   ・CaboChaAnalyzer().parse(text) は解析結果（木構造）を返し、木は
#       - chunk: 文節（名詞句や用言句に相当）  … 依存先インデックス: chunk.link 等を持つ
#       - token: 形態素（表層・品詞・原形など）… token.surface, token.feature 等を持つ
#     という二層構造で走査できる。
#   ・print(token) は人間可読な文字列表現（「表層\tfeature,feature,...」）を出力する。
#     機械処理では token.surface / token.feature を直接参照するのが堅牢。
#   ・環境注意:
#       - cabocha 本体と辞書（MeCab/IPADIC等）が必要。pip だけでなく OS パッケージの導入が要る場合がある。
#       - 解析の再現性は背後の辞書とバージョンに依存するため、実験では使用辞書を固定・記録すること。
# -----------------------------------------------------------------------------------

# Cabochaの利用 (Tokenの取得)
from cabocha.analyzer import CaboChaAnalyzer

# 解析器の生成
# - 追加オプション未指定の場合、環境既定の形態素解析器・辞書が使われる
#   （MeCab が見つからない/辞書不一致の場合はエラーになり得る）
analyzer = CaboChaAnalyzer()

# 解析対象文（短い日常文。文節と形態素の両方の分割が素直に出やすい例）
tree = analyzer.parse("今日はいい天気ですね")

# 解析結果の走査
# - 第一ループ: 文節（chunk）単位で走査
# - 第二ループ: 各文節に含まれる形態素トークン（token）を走査
#   例: chunk.link    -> 係り先の文節インデックス（-1 は根）
#       token.surface -> 表層形
#       token.feature -> 品詞・原形・読み等のカンマ区切り列
for chunk in tree:
    # 参考: 文節の依存先を見たい場合（出力例を増やしたくないのでコメント）
    # print(f'>> chunk -> link:{chunk.link}, score:{chunk.score}')
    for token in chunk:
        # 可視確認用の簡易表示（表層 + feature 列の文字列表現）
        print(token)

        # 構造的に扱う場合の例（必要に応じて利用; ここでは出力は増やさない）
        # s = token.surface
        # f = token.feature  # "品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音"
        # print(s, f, sep='\t')

Token("今日")
Token("は")
Token("いい")
Token("天気")
Token("です")
Token("ね")


In [2]:
# リスト 3.1.2
# Cabochaの利用 (チャンクを先頭からたどる)
# -----------------------------------------------------------------------------------
# 目的:
#   ・係り受け木（tree）から「文節（chunk）リスト」を取得し、
#     先頭チャンク（文頭に位置する文節）と、そのチャンクが係る先（依存先）の
#     チャンクを確認する最小例を示す。
#
# 理論メモ:
#   ・CaboCha の出力は「文節（chunk）」同士の依存関係で表現される有向木（係り受け木）。
#     - 各 chunk は「どの chunk に係るか」を示す **link（依存先のインデックス）** を持つ。
#     - 多くのラッパーでは、`chunk.next_link` が「依存先チャンク（= link で参照される先）」を指す
#       オブジェクト参照として提供される（根に到達すると None になる実装が多い）。
#   ・`tree.chunks` は文中の文節列（通常は出現順）を返す。インデックスは 0 起点。
#   ・「係り受けの向き」に注意:
#       “現在の文節 → 依存先（主辞側の文節）” へ矢印が伸びる（右向きになることが多い）。
#       つまり `next_link` は「自分が係っている先」を辿るので、連続適用で根へ到達する。
#   ・備考: 依存先が存在しない（根）の場合、`chunk.link == -1` かつ `chunk.next_link is None`
#     のような表現になる（実装依存）。その際はさらに先へは辿れない。
#
# 使い方メモ:
#   ・このスニペットは直前のリスト（3.1.1）で作成した `tree = analyzer.parse(...)` が
#     既に存在している前提で動作する（同じノートブック/スクリプトの継続）。
#   ・実運用時には None ガードを入れて安全に辿ること（下のコメント参照）。
# -----------------------------------------------------------------------------------

chunks = tree.chunks  # 文中の文節列（Chunk の配列/リスト）。通常は出現順に並ぶ。
start_chunk = chunks[0]  # 文頭（先頭）の文節。係り受けの起点として観察しやすい。
print(
    "開始チャンク: ", start_chunk
)  # 表示は実装依存（例: <Chunk 0 link=3 ...> 等の簡易表現）

next_chunk = (
    start_chunk.next_link
)  # 先頭文節が「係っている先（依存先）」の文節オブジェクト。
print("次のチャンク: ", next_chunk)  # 根に向かって 1 ステップだけ辿った結果を可視化。

# 参考: 根まで辿るイメージ（安全のため None チェックを入れる。出力が増えるので実行はしない）
# cur = start_chunk
# while cur is not None:
#     print(cur)
#     cur = cur.next_link  # 依存先へ進む（根に到達すると None）

開始チャンク:  Chunk("今日は")
次のチャンク:  Chunk("天気ですね")


In [3]:
# リスト 3.1.3
# Cabochaの利用 (チャンクを最後からたどる)
# -----------------------------------------------------------------------------------
# 目的:
#   ・係り受け木（tree）から文末側（最後の文節 = 右端のチャンク）を取得し、
#     そのチャンクに「係ってくる側」（= 依存している側）の文節集合を確認する。
#
# 理論メモ:
#   ・CaboCha の依存表現は「文節（chunk）」同士の有向木。
#     - 各 chunk は「自分が係る先」を整数インデックスで持つ（通常は chunk.link / ラッパーでは next_link 参照）。
#     - 一方で「自分に係ってくる元」は 0 個以上あるため **集合**（リスト）で表現されることが多い。
#       本ラッパーの `prev_links` は、当該チャンクへ入ってくる辺（依存元チャンク）を列として返す想定。
#   ・`chunks[-1]` は最終チャンク（右端）の文節。日本語の係り受けは右方向に流れることが多く、
#     文末チャンクが根（link == -1）になるケースが典型だが、文や実装により例外もあり得る。
#   ・print の結果はラッパーの `__str__` 実装に依存する（例: `<Chunk i link=j ...>` や `[...]` 形式）。
#
# 使い方メモ:
#   ・直前のリスト（3.1.1/3.1.2）で作成した `tree` と `chunks` が存在している前提で動作する。
#   ・`prev_links` は通常リスト（0 個以上）なので、そのまま print すると配列表現になる。
#     機械処理する場合は for で回して個々のチャンクにアクセスする（ここでは出力増を避け実行しない）。
# -----------------------------------------------------------------------------------

end_chunk = chunks[-1]  # 右端（文末側）のチャンクを取得。根であることが多い。
print("最終チャンク: ", end_chunk)  # 例: <Chunk N link=-1 ...>（実装依存の可読表示）

prev_chunk = (
    end_chunk.prev_links
)  # このチャンクへ「係ってくる」文節の集合（通常は list[Chunk]）
print(
    "一つ前のチャンク: ", prev_chunk
)  # リスト表現で可視化される。単数/複数/空の可能性がある点に注意。

# 参考: 依存元を個別に見たい場合（出力が増えるため実行はしない）
# for src in end_chunk.prev_links or []:
#     print('依存元チャンク: ', src)

# 参考: 文末から逆向きに木をたどる（BFS/DFS）例（疑似; 実行はしない）
# visited = set()
# stack = list(end_chunk.prev_links or [])
# while stack:
#     cur = stack.pop()
#     if id(cur) in visited: continue
#     visited.add(id(cur))
#     # print(cur)
#     stack.extend(cur.prev_links or [])

最終チャンク:  Chunk("天気ですね")
一つ前のチャンク:  [Chunk("今日は"), Chunk("いい")]
